# Simulation

## Load package

In [1]:
using CSV, DataFrames
using JuMP, HiGHS
using Statistics
include(joinpath("..","src","tep.jl"))


tep (generic function with 1 method)

## Load data

In [2]:
nodes = CSV.read(joinpath("..","data","NordicBaltic","nodes.csv"),DataFrame)
generators = CSV.read(joinpath("..","data","NordicBaltic","generators.csv"),DataFrame)
vres = CSV.read(joinpath("..","data","NordicBaltic","vres.csv"),DataFrame)
kperiods = CSV.read(joinpath("..","data","NordicBaltic","kperiods.csv"),DataFrame)
fullcac = CSV.read(joinpath("..","data","NordicBaltic","fullcac.csv"),DataFrame)
fullcdc = CSV.read(joinpath("..","data","NordicBaltic","fullcdc.csv"),DataFrame)
areas = CSV.read(joinpath("..","data","NordicBaltic","areas.csv"),DataFrame)
slacks = [1,2]

2-element Vector{Int64}:
 1
 2

## Parameters

Line related:
- "hvac fixed cost", costacf = 278.2e6
- "hvac variable cost", costacv = 3091.29
- "hvdc fixed cost", costdcf = 1376.3e6
- "hvdc variable cost", costdcv = 2245.11
- "maximum capacity for new line", maxcap = 6000

Others:
- "Base power", Sbase = 100
- "Big M", bigM = 10000
- "MIP gap", epsilon = 0.005

Planning horizon:
- "planning horizon", years = 20

Rates:
- "discount rate", d = 0.05
- "annual load growth rate", load_rate = 0.02
- "annual generation growth rate", gen_rate = 0.01
- "annual VREs growth rate", vre_rate = 0.05
- "intra-zonal reserve requirement ratio", beta = 0.67

## Energy only

In [3]:
CBEO = tep(areas,generators,vres,nodes,kperiods,fullcac,fullcdc,slacks;
years = 10,
mip_gap = 0.005,
CAP_max = 6000,
ac_fixed_cost = 278.2e6,
dc_fixed_cost = 1376.3e6,
ac_var_cost = 3091.29,
dc_var_cost = 2245.11,
d = 0.05,
load_rate = 0.02,
gen_rate = 0.01,
vre_rate = 0.05,
bigM = 10000,
Sbase = 100,
beta = 0.67,
reserve_exchange = false,
budget_based = false,
);
BBEO = tep(areas,generators,vres,nodes,kperiods,fullcac,fullcdc,slacks;
years = 10,
mip_gap = 0.005,
CAP_max = 6000,
ac_fixed_cost = 278.2e6,
dc_fixed_cost = 1376.3e6,
ac_var_cost = 3091.29,
dc_var_cost = 2245.11,
d = 0.05,
load_rate = 0.02,
gen_rate = 0.01,
vre_rate = 0.05,
bigM = 10000,
Sbase = 100,
beta = 0.67,
reserve_exchange = false,
budget_based = true,
);

## Exchange of reserve

In [4]:
CBRX = tep(areas,generators,vres,nodes,kperiods,fullcac,fullcdc,slacks;
years = 10,
mip_gap = 0.005,
CAP_max = 6000,
ac_fixed_cost = 278.2e6,
dc_fixed_cost = 1376.3e6,
ac_var_cost = 3091.29,
dc_var_cost = 2245.11,
d = 0.05,
load_rate = 0.02,
gen_rate = 0.01,
vre_rate = 0.05,
bigM = 10000,
Sbase = 100,
beta = 0.67,
reserve_exchange = true,
budget_based = false,
);
BBRX = tep(areas,generators,vres,nodes,kperiods,fullcac,fullcdc,slacks;
years = 10,
mip_gap = 0.005,
CAP_max = 6000,
ac_fixed_cost = 278.2e6,
dc_fixed_cost = 1376.3e6,
ac_var_cost = 3091.29,
dc_var_cost = 2245.11,
d = 0.05,
load_rate = 0.02,
gen_rate = 0.01,
vre_rate = 0.05,
bigM = 10000,
Sbase = 100,
beta = 0.67,
reserve_exchange = true,
budget_based = true,
);

## Cost breakdown

In [5]:
DataFrame([CBEO.Obj CBRX.Obj BBEO.Obj BBRX.Obj;
sum(CBEO.TCostEn) sum(CBRX.TCostEn) sum(BBEO.TCostEn) sum(BBRX.TCostEn);
sum(CBEO.TCostAs) sum(CBRX.TCostAs) sum(BBEO.TCostAs) sum(BBRX.TCostAs);
sum(CBEO.TCostEns) sum(CBRX.TCostEns) sum(BBEO.TCostEns) sum(BBRX.TCostEns);
sum(CBEO.TCostIntraInv) sum(CBRX.TCostIntraInv) sum(BBEO.TCostIntraInv) sum(BBRX.TCostIntraInv);
sum(CBEO.TCostCrossInv) sum(CBRX.TCostCrossInv) sum(BBEO.TCostCrossInv) sum(BBRX.TCostCrossInv)],:auto) 

Row,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,2.28805e10,2.28805e10,2.08964e10,2.072e10
2,1.44621e10,1.44621e10,1.28731e10,1.28275e10
3,1.23676e10,1.23676e10,1.25585e10,1.23604e10
4,0.0,0.0,0.0,0.0
5,8.73828e8,8.73828e8,9.56445e9,1.23274e10
6,0.0,0.0,1.66716e10,1.66347e10


## HVAC Investment Decisions

In [22]:
DataFrame([nodes[fullcac.f_node,:name] nodes[fullcac.t_node,:name] [sum(CBEO.cap_ac,dims=2) sum(CBRX.cap_ac,dims=2) sum(BBEO.cap_ac,dims=2) sum(BBRX.cap_ac,dims=2)] ],["f_node","t_node","CBEO","CBRX","BBEO","BBRX"])

Row,f_node,t_node,CBEO,CBRX,BBEO,BBRX
,Any,Any,Any,Any,Any,Any
1,Denmark,Finland,6000.0,6000.0,3488.33,5797.81
2,Denmark,Sweden,0.0,0.0,4408.27,6000.0
3,Denmark,Norway,6000.0,6000.0,6000.0,6000.0
4,Finland,Sweden,0.0,0.0,5515.77,3516.19
5,Finland,Norway,0.0,0.0,2194.51,6000.0
6,Sweden,Norway,0.0,0.0,4521.45,6000.0
7,Estonia,Latvia,0.0,0.0,6000.0,4935.3
8,Estonia,Lithuania,689.835,689.835,3781.72,3791.02
9,Latvia,Lithuania,0.0,0.0,4475.28,0.0


## HVDC Investment Decisions

In [27]:
DataFrame([nodes[fullcdc.f_node,:name] nodes[fullcdc.t_node,:name] sum(CBEO.cap_dc,dims=2) sum(CBRX.cap_dc,dims=2) sum(BBEO.cap_dc,dims=2) sum(BBRX.cap_dc,dims=2)],["f_node","t_node","CBEO","CBRX","BBEO","BBRX"])

Row,f_node,t_node,CBEO,CBRX,BBEO,BBRX
,Any,Any,Any,Any,Any,Any
1,Denmark,Estonia,0.0,0.0,4889.31,6000.0
2,Denmark,Finland,0.0,0.0,0.0,0.0
3,Denmark,Latvia,0.0,0.0,6000.0,0.0
4,Denmark,Lithuania,0.0,0.0,4604.82,0.0
5,Denmark,Sweden,0.0,0.0,6000.0,0.0
6,Denmark,Norway,0.0,0.0,6000.0,0.0
7,Estonia,Finland,0.0,0.0,6000.0,6000.0
8,Estonia,Latvia,0.0,0.0,6000.0,6000.0
9,Estonia,Lithuania,0.0,0.0,0.0,0.0
